In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config2 import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Solok,ID,6/5/21 20:28,-0.7918,100.6593,68.50,87,4,2.30
1,1,Hasaki,JP,6/5/21 20:28,35.7333,140.8333,66.60,82,100,9.35
2,2,Albany,US,6/5/21 20:28,42.6001,-73.9662,91.45,44,47,1.01
3,3,Zhuhai,CN,6/5/21 20:28,22.2769,113.5678,81.05,83,20,3.00
4,4,Kavaratti,IN,6/5/21 20:28,10.5669,72.6420,82.42,76,100,13.62


In [3]:
# Since data we use for any mapping must be either an integer or a floating-point decimal number, check the city data
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature option 1.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

## List comprehension - Option 2
## [max(temp, 0) for temp in max_temp]

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
#heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Let's start to build a search for the locations to travel

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Zhuhai,CN,6/5/21 20:28,22.2769,113.5678,81.05,83,20,3.00
4,4,Kavaratti,IN,6/5/21 20:28,10.5669,72.6420,82.42,76,100,13.62
5,5,Butaritari,KI,6/5/21 20:28,3.0707,172.7902,82.27,70,30,5.50
6,6,Hithadhoo,MV,6/5/21 20:28,-0.6000,73.0833,82.38,75,100,12.48
8,8,Kapaa,US,6/5/21 20:28,22.0752,-159.3190,84.18,69,20,5.01


In [13]:
preferred_cities_df.count()

City_ID       179
City          179
Country       179
Date          179
Lat           179
Lng           179
Max Temp      179
Humidity      179
Cloudiness    179
Wind Speed    179
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Zhuhai,CN,81.05,22.2769,113.5678,
4,Kavaratti,IN,82.42,10.5669,72.6420,
5,Butaritari,KI,82.27,3.0707,172.7902,
6,Hithadhoo,MV,82.38,-0.6000,73.0833,
8,Kapaa,US,84.18,22.0752,-159.3190,
10,Bengkulu,ID,75.49,-3.8004,102.2655,
12,Arraial Do Cabo,BR,75.16,-22.9661,-42.0278,
15,Roma,US,86.09,43.2128,-75.4557,
19,Rikitea,PF,75.42,-23.1203,-134.9692,
26,Tual,ID,81.16,-5.6667,132.7500,


In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Zhuhai,CN,81.05,22.2769,113.5678,Nanyang Seascape Hotel Zhuhai Hotel
4,Kavaratti,IN,82.42,10.5669,72.6420,Hotels in Lakshadweep Islands
5,Butaritari,KI,82.27,3.0707,172.7902,Isles Sunset Lodge
6,Hithadhoo,MV,82.38,-0.6000,73.0833,Scoop Guest House
8,Kapaa,US,84.18,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Bengkulu,ID,75.49,-3.8004,102.2655,Grage Hotel Bengkulu
12,Arraial Do Cabo,BR,75.16,-22.9661,-42.0278,Pousada Porto Praia
15,Roma,US,86.09,43.2128,-75.4557,Wingate By Wyndham Rome
19,Rikitea,PF,75.42,-23.1203,-134.9692,Pension Maro'i
26,Tual,ID,81.16,-5.6667,132.7500,Grand Vilia Hotel


In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [23]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [24]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))